# Importing libraries

In [2]:
import io
import requests
import docx
import json

# FAQ Data Inside the files - (Namings <span style="color:red">Q</span> and <span style="color:red">Ans</span> is just for showcasing here, otherwise Question is in bold in the files)

Q- Java - RuntimeError: Java gateway process exited before sending its port number

Ans- Shows java_home is not set on the notebook log
https://sparkbyexamples.com/pyspark/pyspark-exception-java-gateway-process-exited-before-sending-the-driver-its-port-number/
https://twitter.com/drkrishnaanand/status/1765423415878463839

Q- Spark BigQuery connector Automatic configuration

Ans- While creating a SparkSession using the config spark.jars.packages as com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.23.2
spark = SparkSession.builder.master('local').appName('bq').config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.23.2").getOrCreate() 

automatically downloads the required dependency jars and configures the connector, removing the need to manage this dependency. More details available here

.
.
.
.

# Data Files

In [3]:
# There are three different files corresponding to three different courses, and we have file ids here as these files are stored on google drive
faq_documents = {
    'data-engineering-zoomcamp': '19bnYs80DwuUimHM65UV3sylsCn2j1vziPOwzBwQrebw',
    'machine-learning-zoomcamp': '1LpPanc33QJJ6BSsyxVg-pWNMplal84TdZtq10naIhD8',
    'mlops-zoomcamp': '12TlBfhIiKtyBv8RnsoJR6F72bkPDGEvPOItJIxaEzE0',
}

# Structure of data inside the data files

### <span style="color:red">SECTION ---> QUESTION ---> ANSWER </span>

# Structure of O/p we want

<span style="color:orange">
<b>{
    <b  style="color:red">"COURSE"</b>: "COURSE HEADING OR THE COURSE NAME FROM WHICH Q/A ARE TO BE MADE", <br><br>
     <b  style="color:red">"DOCUMENTS"</b>: <span style="color:green">[</span>  <br>
      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:blue"> <! Document-2>
          {</span> <br>
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"TEXT1": "ANSWER TO THE QUESTION",<br><br>
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"SECTION1": "SECTION NAME",<br><br>
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"QUESTION1": "QUESTION TO BE ASKED"<br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <span style="color:blue">}, </span>
    <br>
    <br>
    <br>
    <! Document-2>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:blue"> {</span> <br>
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"TEXT2": "ANSWER TO THE QUESTION",<br><br>
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"SECTION2": "SECTION NAME",<br><br>
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"QUESTION2": "QUESTION TO BE ASKED"<br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <span style="color:blue">}, </span>  <br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <span style="color:green">]</span> <br>
    }
</span>

# Reading the data files and forming into the above structure output

In [4]:
# Function to read the data from whole data file and create a json of Q/A
def read_faq(file_id):
    url = f'https://docs.google.com/document/d/{file_id}/export?format=docx'
    response = requests.get(url)
    
    # Since the response will be in Bytes we need to make a relevant conversion
    # Also, we are using the docx package since we have files in the format of .docx, and docx package is used for reading .docx files
    with io.BytesIO(response.content) as f_in:
        doc = docx.Document(f_in)

    # To store different sets of documents each having- TEXT, SECTION, QUESTION in a form of dictionary as can be seen from above
    document_set = []

    # The font styles of the section heading and question in the document
    # In the document, Section is having bigger font than Question
    section_heading_style = 'heading 1'
    question_heading_style = 'heading 2'

    # Variable to store section, question, answer 
    section_title = ''
    question_title = ''
    answer_text_so_far = ''

    # Looping through each line in the doc
    for p in doc.paragraphs:
        # Knowing the font style of each line of the doc- Normal, Heading1, Heading2 as can be seen in Microsoft word
        style = p.style.name.lower()
        # Getting the text of the line
        p_text = p.text
        p_text = p_text.strip() # Removing spaces from either sides of the line
        p_text = p_text.strip('\uFEFF') # This unicode returns the error so it is good to remove it and it occurs at the beginning of the file 

        # When it's a blank line
        if len(p_text) == 0:
            continue

        # When it's a Section line
        if style == section_heading_style:
            section_title = p_text
            continue

        # When it's a Question line- So thiss is checking the new question, and if it's a new question then whatever values for 
        # the previous question answer were there we are appending in the questions variable
        if style == question_heading_style:
            answer_text_so_far = answer_text_so_far.strip()
            # Checking if it's not the starting of the parsing when all the section_title,
            # question_title and answer to any of the question has not been captured
            if answer_text_so_far != '' and section_title != '' and question_title != '':
                # Storing previous Q/A values
                document_set.append({
                    'text': answer_text_so_far,
                    'section': section_title,
                    'question': question_title,
                })
                answer_text_so_far = ''
    
            question_title = p_text
            continue

        # When the text is not of the Section or Question fonts then it is appended to the 'Answer' variable which is "answer_text_so_far"
        answer_text_so_far += '\n' + p_text
    
    # When it's the last line of the document where we won't be having any new question to make the storage for the last Q/A
    answer_text_so_far = answer_text_so_far.strip()
    if answer_text_so_far != '' and section_title != '' and question_title != '':
        document_set.append({
            'text': answer_text_so_far,
            'section': section_title,
            'question': question_title,
        })

    return document_set
    

In [5]:
documents = []

for course, file_id in faq_documents.items():
    print(course)
    course_documents = read_faq(file_id)
    documents.append({'course': course, 'documents': course_documents})

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp


In [12]:
documents[0].keys()

dict_keys(['course', 'documents'])

# Storing the Output in a JSON file

In [7]:
# Creating a json file to store the data from all the data files
with open('data/documents.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [8]:
# Printing one of the JSON element
!head data/documents.json

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "The purpose of this document is to capture frequently asked\ntechnical question\nThe next cohort starts in Jan 2025. More inFo at DTC Article.\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {
